# Amazon Bedrock boto3 Setup

> **Pre-requisites**:
> - [Python 3.9](https://www.python.org/downloads/)

> **Setup environment**:
> - `python3.9 -m venv .venv`
> - `source .venv/bin/activate`
> - `pip install -r requirements.txt`
> ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

- We will use AWS SDK directly to interact with AWS services.

### AWS Services

In [1]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

  Using cached boto3-1.34.5-py3-none-any.whl.metadata (6.6 kB)
  Using cached awscli-1.32.5-py3-none-any.whl.metadata (11 kB)
  Using cached botocore-1.34.5-py3-none-any.whl.metadata (5.6 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.9.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl.metadata (8.6 kB)
Using cached boto3-1.34.5-py3-none-any.whl (139 kB)
Using cached awscli-1.32.5-py3-none-any.whl (4.3 MB)
Using cached botocore-1.34.5-py3-none-any.whl (11.9 MB)
Using cached PyYAML-6

### agents

In [2]:
%pip install --quiet \
    xmltodict==0.13.0  \
    duckduckgo-search  \
    yfinance  \
    pandas_datareader  \
    langchain_experimental \
    google-search-results

Note: you may need to restart the kernel to use updated packages.


### entities

In [3]:
%pip install --quiet beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


### image

In [4]:
%pip install --quiet "pillow>=9.5,<10"


Note: you may need to restart the kernel to use updated packages.


### gaurdrails

%%brew update
%%brew install gcc


#### Uninstall unwanted packages

In [5]:
!python -m pip uninstall langchain -y
%pip uninstall langchain_experimental -y
%pip uninstall langchain-0.0.352 -y
%pip uninstall langchain_experimental -y

Found existing installation: langchain 0.0.352
Uninstalling langchain-0.0.352:
  Successfully uninstalled langchain-0.0.352
Found existing installation: langchain-experimental 0.0.47
Uninstalling langchain-experimental-0.0.47:
  Successfully uninstalled langchain-experimental-0.0.47
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### nemoguardrails

In [6]:
%pip install -qU --no-cache-dir nemoguardrails==0.5.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-core 2.14.5 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


%pip install -qU "faiss-cpu>=1.7,<2" \
                      "pypdf>=3.8,<4" \
                      "ipywidgets>=7,<8"

In [7]:
%pip install -qU "langchain>=0.0.251" 
%pip install -qU "langchain_experimental>=0.0.30"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nemoguardrails 0.5.0 requires langchain==0.0.251, but you have langchain 0.0.352 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Restart kernel

In [8]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup AWS Credentials

1.  Create a new IAM user with `local code` access to Amazon Bedrock resources. 
    *AmazonBedrockFullAccess* policy will provide all the required permissions to the user.
    *AmazonS3FullAccess* policy will provide access to S3 bucket.

    [Create IAM user](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_users_create.html#id_users_create_console)

    ![IAM Image](https://raw.githubusercontent.com/Dinuda/BedrockBoto3Setup/main/images/confuser.png?raw=true)

2. Create a new AWS profile
    ```bash
    aws configure --profile bedrockuser

    AWS Access Key ID [None]: <access_key_id>
    AWS Secret Access Key [None]: <secret_access_key>
    Default region name [None]: <region>
    Default output format [None]: text
    ```

3. Create new AWS service role for Bedrock
   
    [Create service role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_create_for-service.html#roles-creatingrole-service-console)
    
    ![IAM Image](https://raw.githubusercontent.com/Dinuda/BedrockBoto3Setup/main/images/servrole.png?raw=true)
   
    -- Trust Policy
    ```
    {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "AWS": "arn:aws:iam::<userid>:role/service-role/AgentFineTuningServiceRole",
                    "Service": [
                        "sagemaker.amazonaws.com",
                        "bedrock.amazonaws.com",
                        "events.amazonaws.com"
                    ]
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    ```

4. Copy the `Role ARN`

### Create client

In [10]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))


os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "bedrockuser"
os.environ["BEDROCK_ASSUME_ROLE"] = ""  # E.g. "arn:aws:..."

boto3_bedrock = boto3.client('bedrock')


#### Validate the connection




In [11]:
boto3_bedrock.list_foundation_models()


{'ResponseMetadata': {'RequestId': '1756df5b-8b05-4665-acb2-4d74ed0eaeeb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 21 Dec 2023 12:31:08 GMT',
   'content-type': 'application/json',
   'content-length': '17836',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1756df5b-8b05-4665-acb2-4d74ed0eaeeb'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

### Invoke the API

In [12]:
bedrock_runtime = boto3.client('bedrock-runtime')


#### Anthropic Claude

In [13]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Write me a blog about making strong business decisions as a leader.

Assistant:
"""


In [14]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


 Here is a draft blog post on making strong business decisions as a leader:

Making Strong Business Decisions as a Leader

As a leader, one of your most important responsibilities is making thoughtful business decisions that help drive your company forward. However, decision making can be challenging, as there are many factors to weigh and outcomes that are uncertain. Here are some tips for making strong choices that empower your business:

Gather Input from Multiple Perspectives
When facing an important decision, don't rely solely on your own perspective. Consult others inside and outside your organization who have relevant expertise. Get input from teams impacted by the decision as well as those who will implement it. Understanding differing viewpoints will lead to well-rounded analysis of options and consequences. 

Consider Both Short and Long-Term Impacts
While short-term gains may be tempting, focus first on decisions that align with your company's long-term strategy and vision. 